# Microdust Project Template Builder

#### The project is consisted of two parts: Correlation studies and Prediction
#### I have gathered the data from multiple sources including the public open data forum, non-profit organization and API
#### For easier access and management of the codes, I have separated the preprocessing codes from analysis and prediction codes
#### This notebook includes the steps from importing data to creating the template for the analysis, which is covered in the other notebook

<img src = "http://image.ytn.co.kr/general/jpg/2017/0506/201705061032135481_t.jpg">

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
from datetime import timedelta

# A: Data preparation for Correlation Analysis

## 1. Prepare the Air Pollution Data of major cities in China
* Due to the Westerlies, Chinese air pollution partially affects the air quality of Korea.
* stateair.net provides the historical data of air quality in major Chinese cities
* I am going to use '15 and '16 data for the training and use '17 data for the testing

In [2]:
beijing_15 = pd.read_csv('china_air/beijing_2015_HourlyPM25_created20160201.csv', encoding='ISO-8859-1')

In [3]:
beijing_16 = pd.read_csv('china_air/beijing_2016_HourlyPM25_created20170201.csv', encoding='ISO-8859-1')

In [4]:
beijing_17 = pd.read_csv('china_air/beijing_2017_HourlyPM25_created20170803.csv', encoding='ISO-8859-1')

In [5]:
beijing_15.head()

,A fact sheet with definitions and metadata for this dataset can be found at http://www.stateair.net/web/historical/1/1.html.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,The U.S. Department of State Data Use Statemen...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Site,Parameter,Date (LST),Year,Month,Day,Hour,Value,Unit,Duration,QC Name
3,Beijing,PM2.5,1/1/2015 0:00,2015,1,1,0,22,µg/m³,1 Hr,Valid
4,Beijing,PM2.5,1/1/2015 1:00,2015,1,1,1,9,µg/m³,1 Hr,Valid


### The D.F is not well structured. Let's make few adjustments:
    * Create relevant columns
    * Delete first 2 rows with NaN
    * Check out the values where 'QC Name' is not 'Valid'

In [6]:
beijing_15.iloc[:, 10].value_counts()

Valid      8671
Missing      89
QC Name       1
Name: Unnamed: 10, dtype: int64

#### About 1% of the data is filled with 'Missing' and they are filled with '-999'

In [7]:
beijing_15[beijing_15.iloc[:, 10]=='Missing']

,A fact sheet with definitions and metadata for this dataset can be found at http://www.stateair.net/web/historical/1/1.html.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
349,Beijing,PM2.5,1/15/2015 10:00,2015,1,15,10,-999,µg/m³,1 Hr,Missing
592,Beijing,PM2.5,1/25/2015 13:00,2015,1,25,13,-999,µg/m³,1 Hr,Missing
738,Beijing,PM2.5,1/31/2015 15:00,2015,1,31,15,-999,µg/m³,1 Hr,Missing
739,Beijing,PM2.5,1/31/2015 16:00,2015,1,31,16,-999,µg/m³,1 Hr,Missing
740,Beijing,PM2.5,1/31/2015 17:00,2015,1,31,17,-999,µg/m³,1 Hr,Missing
811,Beijing,PM2.5,2/3/2015 16:00,2015,2,3,16,-999,µg/m³,1 Hr,Missing
998,Beijing,PM2.5,2/11/2015 11:00,2015,2,11,11,-999,µg/m³,1 Hr,Missing
999,Beijing,PM2.5,2/11/2015 12:00,2015,2,11,12,-999,µg/m³,1 Hr,Missing
1000,Beijing,PM2.5,2/11/2015 13:00,2015,2,11,13,-999,µg/m³,1 Hr,Missing
1045,Beijing,PM2.5,2/13/2015 10:00,2015,2,13,10,-999,µg/m³,1 Hr,Missing


#### I am going to fill in the missing data with the previously observed values(ffill)

In [8]:
def edit_china_df(df_list):
    for df in df_list:
        #Create the columns and drop unnecessary rows
        df.columns = df.iloc[2, :]
        df.drop(df.index[0:3], inplace = True)
        #Make the date_time column to use as an index
        df['date_time'] = df['Year'].astype(str) + "-" + df['Month'].astype(str) + "-" + df['Day'].astype(str) + "-" + df["Hour"].astype(str)
        #Replace the confusing values to NaN and execute fillna
        df['Value'].replace('-999',np.nan, inplace = True)
        df['Value'].fillna(method = 'ffill', axis = 0, inplace = True)
        print (df.shape)
        print (df.head(3))
        #Get rid of unrelevant columns
        df.drop(['Site', 'Parameter', 'Date (LST)', 'Year', 'Month', 'Day', 'Hour', 'Unit', 'Duration','QC Name'], axis=1, inplace=True)
        #Check if the DataFrame includes NaN values, which may cause problems in later stage
        print ("Has NaN Values? ", df.isnull().values.any())  

In [9]:
beijing_list = [beijing_15, beijing_16, beijing_17]
edit_china_df(beijing_list)

(8760, 12)
2     Site Parameter     Date (LST)  Year Month Day Hour Value   Unit  \
3  Beijing     PM2.5  1/1/2015 0:00  2015     1   1    0    22  µg/m³   
4  Beijing     PM2.5  1/1/2015 1:00  2015     1   1    1     9  µg/m³   
5  Beijing     PM2.5  1/1/2015 2:00  2015     1   1    2     9  µg/m³   

2 Duration QC Name   date_time  
3     1 Hr   Valid  2015-1-1-0  
4     1 Hr   Valid  2015-1-1-1  
5     1 Hr   Valid  2015-1-1-2  
Has NaN Values?  False
(8784, 12)
2     Site Parameter     Date (LST)  Year Month Day Hour Value   Unit  \
3  Beijing     PM2.5  1/1/2016 0:00  2016     1   1    0   231  µg/m³   
4  Beijing     PM2.5  1/1/2016 1:00  2016     1   1    1   239  µg/m³   
5  Beijing     PM2.5  1/1/2016 2:00  2016     1   1    2   205  µg/m³   

2 Duration QC Name   date_time  
3     1 Hr   Valid  2016-1-1-0  
4     1 Hr   Valid  2016-1-1-1  
5     1 Hr   Valid  2016-1-1-2  
Has NaN Values?  False
(4344, 12)
2     Site Parameter     Date (LST)  Year Month Day Hour Value   Unit  

In [10]:
# Finally, let's change the value to the float type for calculation
beijing_15.rename(columns ={'Value' : 'Value_Beijing'}, inplace = True)
beijing_15['Value_Beijing'] = beijing_15['Value_Beijing'].astype(np.float64)

beijing_16.rename(columns ={'Value' : 'Value_Beijing'}, inplace = True)
beijing_16['Value_Beijing'] = beijing_16['Value_Beijing'].astype(np.float64)

beijing_17.rename(columns ={'Value' : 'Value_Beijing'}, inplace = True)
beijing_17['Value_Beijing'] = beijing_17['Value_Beijing'].astype(np.float64)

beijing_15.tail()

2,Value_Beijing,date_time
8758,133.0,2015-12-31-19
8759,169.0,2015-12-31-20
8760,203.0,2015-12-31-21
8761,212.0,2015-12-31-22
8762,235.0,2015-12-31-23


In [11]:
shanghai_15 = pd.read_csv('china_air/shanghai_2015_HourlyPM25_created20160201.csv', encoding='ISO-8859-1')

In [12]:
shanghai_16 = pd.read_csv('china_air/shanghai_2016_HourlyPM25_created20170201.csv', encoding='ISO-8859-1')

In [13]:
shanghai_17 = pd.read_csv('china_air/shanghai_2017_HourlyPM25_created20170803.csv', encoding='ISO-8859-1')

In [14]:
shanghai_list = [shanghai_15, shanghai_16, shanghai_17]
edit_china_df(shanghai_list)

(8760, 12)
2      Site Parameter     Date (LST)  Year Month Day Hour Value   Unit  \
3  Shanghai     PM2.5  1/1/2015 0:00  2015     1   1    0    32  µg/m³   
4  Shanghai     PM2.5  1/1/2015 1:00  2015     1   1    1    40  µg/m³   
5  Shanghai     PM2.5  1/1/2015 2:00  2015     1   1    2    37  µg/m³   

2 Duration QC Name   date_time  
3     1 Hr   Valid  2015-1-1-0  
4     1 Hr   Valid  2015-1-1-1  
5     1 Hr   Valid  2015-1-1-2  
Has NaN Values?  False
(8784, 12)
2      Site Parameter     Date (LST)  Year Month Day Hour Value   Unit  \
3  Shanghai     PM2.5  1/1/2016 0:00  2016     1   1    0    60  µg/m³   
4  Shanghai     PM2.5  1/1/2016 1:00  2016     1   1    1    65  µg/m³   
5  Shanghai     PM2.5  1/1/2016 2:00  2016     1   1    2    57  µg/m³   

2 Duration QC Name   date_time  
3     1 Hr   Valid  2016-1-1-0  
4     1 Hr   Valid  2016-1-1-1  
5     1 Hr   Valid  2016-1-1-2  
Has NaN Values?  False
(4344, 12)
2      Site Parameter     Date (LST)  Year Month Day Hour Value

In [15]:
shanghai_15.rename(columns ={'Value' : 'Value_Shanghai'}, inplace = True)
shanghai_15['Value_Shanghai'] = shanghai_15['Value_Shanghai'].astype(np.float64)

shanghai_16.rename(columns ={'Value' : 'Value_Shanghai'}, inplace = True)
shanghai_16['Value_Shanghai'] = shanghai_16['Value_Shanghai'].astype(np.float64)

shanghai_17.rename(columns ={'Value' : 'Value_Shanghai'}, inplace = True)
shanghai_17['Value_Shanghai'] = shanghai_17['Value_Shanghai'].astype(np.float64)

shanghai_16.tail()


2,Value_Shanghai,date_time
8782,43.0,2016-12-31-19
8783,40.0,2016-12-31-20
8784,35.0,2016-12-31-21
8785,39.0,2016-12-31-22
8786,44.0,2016-12-31-23


In [16]:
shenyang_15 = pd.read_csv('china_air/shenyang_2015_HourlyPM25_created20160201.csv', encoding='ISO-8859-1')
shenyang_16 = pd.read_csv('china_air/shenyang_2016_HourlyPM25_created20170201.csv', encoding='ISO-8859-1')
shenyang_17 = pd.read_csv('china_air/shenyang_2017_HourlyPM25_created20170803.csv', encoding='ISO-8859-1')

In [17]:
shenyang_list = [shenyang_15, shenyang_16, shenyang_17]
edit_china_df(shenyang_list)

(8760, 12)
2      Site Parameter     Date (LST)  Year Month Day Hour Value   Unit  \
3  Shenyang     PM2.5  1/1/2015 0:00  2015     1   1    0    37  µg/m³   
4  Shenyang     PM2.5  1/1/2015 1:00  2015     1   1    1    26  µg/m³   
5  Shenyang     PM2.5  1/1/2015 2:00  2015     1   1    2    30  µg/m³   

2 Duration QC Name   date_time  
3     1 Hr   Valid  2015-1-1-0  
4     1 Hr   Valid  2015-1-1-1  
5     1 Hr   Valid  2015-1-1-2  
Has NaN Values?  False
(8784, 12)
2      Site Parameter     Date (LST)  Year Month Day Hour Value   Unit  \
3  Shenyang     PM2.5  1/1/2016 0:00  2016     1   1    0   218  µg/m³   
4  Shenyang     PM2.5  1/1/2016 1:00  2016     1   1    1   170  µg/m³   
5  Shenyang     PM2.5  1/1/2016 2:00  2016     1   1    2    93  µg/m³   

2 Duration QC Name   date_time  
3     1 Hr   Valid  2016-1-1-0  
4     1 Hr   Valid  2016-1-1-1  
5     1 Hr   Valid  2016-1-1-2  
Has NaN Values?  False
(4344, 12)
2      Site Parameter     Date (LST)  Year Month Day Hour Value

In [18]:
shenyang_15.rename(columns ={'Value' : 'Value_Shenyang'}, inplace = True)
shenyang_15['Value_Shenyang'] = shenyang_15['Value_Shenyang'].astype(np.float64)

shenyang_16.rename(columns ={'Value' : 'Value_Shenyang'}, inplace = True)
shenyang_16['Value_Shenyang'] = shenyang_16['Value_Shenyang'].astype(np.float64)

shenyang_17.rename(columns ={'Value' : 'Value_Shenyang'}, inplace = True)
shenyang_17['Value_Shenyang'] = shenyang_17['Value_Shenyang'].astype(np.float64)

shenyang_17.tail()

2,Value_Shenyang,date_time
4342,51.0,2017-6-30-19
4343,47.0,2017-6-30-20
4344,29.0,2017-6-30-21
4345,62.0,2017-6-30-22
4346,40.0,2017-6-30-23


## 2. Prepare the Temperature Data of South Korea

In [19]:
temp_15 = pd.read_csv('temperature/2015_temp.csv', encoding='euc-kr')
temp_15.name = 'temp_15'

for col in  temp_15.columns[2:]:
    temp_15[col] = pd.to_numeric(temp_15[col], errors='coerce')


In [20]:
temp_16 = pd.read_csv('temperature/2016_temp.csv', encoding='euc-kr')
temp_16.name = 'temp_16'

for col in  temp_16.columns[2:]:
    temp_16[col] = pd.to_numeric(temp_15[col], errors='coerce')

In [21]:
temp_17 = pd.read_csv('temperature/2017_temp.csv', encoding='euc-kr')
temp_17.name = 'temp_17'

for col in  temp_17.columns[2:]:
    temp_17[col] = pd.to_numeric(temp_15[col], errors='coerce')

In [22]:
temp_15.head()

,지점,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,108,2015-01-01 0:00,-6.9,NaN,6.1,290,31.0,1.1,-21.2,1011.6,...,NaN,NaN,2000.0,NaN,NaN,-3.8,-2.7,-1.1,0.1,0.7
1,108,2015-01-01 1:00,-7.4,NaN,4.7,290,33.0,1.2,-20.9,1011.8,...,NaN,NaN,NaN,NaN,NaN,-4.5,-3.2,-1.3,0.1,0.8
2,108,2015-01-01 2:00,-8.0,NaN,4.5,290,35.0,1.2,-20.8,1011.7,...,NaN,NaN,NaN,NaN,NaN,-5.1,-3.7,-1.7,0.0,0.7
3,108,2015-01-01 3:00,-8.4,NaN,3.8,290,37.0,1.2,-20.5,1012.1,...,NaN,NaN,2000.0,4.0,NaN,-5.5,-4.0,-1.9,0.0,0.7
4,108,2015-01-01 4:00,-8.8,NaN,4.9,290,35.0,1.1,-21.5,1012.3,...,NaN,NaN,2000.0,NaN,NaN,-5.9,-4.4,-2.2,0.0,0.7


### The table structure seems to be a lot more reliable compared to the Chinese air data.  Let's validate if the number of the rows is same as the table of Chinese air data.

In [23]:
temp_list = [temp_15, temp_16, temp_17]

In [24]:
for df in temp_list:   
    print (df.name)
    print ("\t The shape: ", df.shape)
    print ("\t Nan values: ", df.isnull().values.any())

temp_15
	 The shape:  (8760, 27)
	 Nan values:  True
temp_16
	 The shape:  (8784, 27)
	 Nan values:  True
temp_17
	 The shape:  (5088, 27)
	 Nan values:  True


#### Oops! We have NaN values in DataFrame, but let's handle this in later stage.
#### We have the same number of row for '15 and '16 but different number for '17, and as we can see from the below dataframe, it's because Korean temperature dataframe includes July Data. 

In [25]:
temp_17[temp_17['일시'].str.contains('2017-07')]

,지점,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),...,운형(운형약어),최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
4344,108,2017-07-01 0:00,22.2,0.0,0.5,180,84.0,22.4,19.3,985.2,...,NaN,10.0,600.0,NaN,1901.0,22.6,23.2,24.3,24.9,24.5
4345,108,2017-07-01 1:00,22.1,0.5,1.9,200,84.0,22.2,19.2,984.6,...,NaN,10.0,500.0,NaN,1901.0,22.3,23.0,24.1,24.8,24.4
4346,108,2017-07-01 2:00,21.9,NaN,0.8,230,83.0,21.7,18.8,984.0,...,NaN,NaN,600.0,NaN,19.0,22.0,22.7,23.9,24.7,24.3
4347,108,2017-07-01 3:00,21.8,NaN,2.0,230,84.0,21.8,18.9,983.4,...,NaN,10.0,600.0,1.0,19.0,22.0,22.6,23.7,24.5,24.2
4348,108,2017-07-01 4:00,21.7,NaN,2.7,250,88.0,22.8,19.6,983.3,...,NaN,10.0,700.0,NaN,19.0,21.6,22.5,23.6,24.4,24.1
4349,108,2017-07-01 5:00,21.4,NaN,3.1,270,85.0,21.6,18.7,983.4,...,NaN,10.0,800.0,NaN,19.0,21.4,22.1,23.4,24.2,24.0
4350,108,2017-07-01 6:00,21.0,NaN,3.8,270,84.0,20.8,18.1,984.1,...,NaN,10.0,900.0,NaN,19.0,21.3,21.8,23.1,24.1,23.9
4351,108,2017-07-01 7:00,21.1,NaN,2.1,270,82.0,20.4,17.8,984.2,...,NaN,10.0,900.0,NaN,19.0,22.1,22.2,23.0,23.9,23.8
4352,108,2017-07-01 8:00,22.2,NaN,2.3,270,75.0,20.0,17.5,984.9,...,NaN,10.0,900.0,NaN,19.0,25.3,24.1,23.2,23.9,23.8
4353,108,2017-07-01 9:00,23.3,NaN,3.7,320,68.0,19.4,17.0,984.7,...,NaN,10.0,1700.0,1.0,19.0,30.3,26.5,23.7,23.9,23.7


#### I will remove the July data from '17 dataframe to match the row size with Chinese air pollution data

In [26]:
temp_17.drop(temp_17.index[4344:], inplace = True, axis = 0)

#### Now it's the same

In [27]:
temp_17.shape

(4344, 27)

### There are unnecessary items in columns.  
### Let's narrow down the columns to the ones we need for analysis and translate the column names to English

In [28]:
temp_15.columns

Index(['지점', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '풍향(16방위)', '습도(%)',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '해면기압(hPa)', '일조(hr)',
       '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)', '전운량(10분위)', '중하층운량(10분위)',
       '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)', '현상번호(국내식)',
       '지면온도(°C)', '5cm 지중온도(°C)', '10cm 지중온도(°C)', '20cm 지중온도(°C)',
       '30cm 지중온도(°C)'],
      dtype='object')

In [29]:
for temp in [temp_15, temp_16, temp_17]:
    temp.drop(['지점', '지면상태(지면상태코드)', '현상번호(국내식)','일조(hr)', '일사(MJ/m2)',
              '운형(운형약어)'], axis=1, inplace = True)
    temp.columns = ['datetime', 'temp','rain_amt','windspeed','wind_direction',
                    'humidity', 'vapor_pressure','dewpoint_temp','spot_pressure',
                    'sea_level_pressure','snow_amt', 'snow_3hrs', 'cloud_amt', 
                    'mid_cloud_amt', 'cloud_height', 'visibility', 'surface_temp',
                    'soil_temp(5cm)', 'soil_temp(10cm)', 'soil_temp(20cm)',
                    'soil_temp(30cm)']
    
temp_15.columns

Index(['datetime', 'temp', 'rain_amt', 'windspeed', 'wind_direction',
       'humidity', 'vapor_pressure', 'dewpoint_temp', 'spot_pressure',
       'sea_level_pressure', 'snow_amt', 'snow_3hrs', 'cloud_amt',
       'mid_cloud_amt', 'cloud_height', 'visibility', 'surface_temp',
       'soil_temp(5cm)', 'soil_temp(10cm)', 'soil_temp(20cm)',
       'soil_temp(30cm)'],
      dtype='object')

* I would like to make a column 'date_time', which I am going to use as an index when I merge multiple DataFrames. 
* There is a minor difference, however, on how the datetime is formatted for '15, '17 and '16.(Below DataFrame shows that 'datetime' column is written in slightly different format.  
* This can create problems when merging the DataFrame later on. So let's make the format identical.

In [30]:
# datetime in '15  and '17 data has a single digit for hour
temp_15.head(2)

,datetime,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,dewpoint_temp,spot_pressure,sea_level_pressure,...,snow_3hrs,cloud_amt,mid_cloud_amt,cloud_height,visibility,surface_temp,soil_temp(5cm),soil_temp(10cm),soil_temp(20cm),soil_temp(30cm)
0,2015-01-01 0:00,-6.9,NaN,6.1,290,31.0,1.1,-21.2,1011.6,1022.8,...,NaN,0.0,0.0,NaN,2000.0,-3.8,-2.7,-1.1,0.1,0.7
1,2015-01-01 1:00,-7.4,NaN,4.7,290,33.0,1.2,-20.9,1011.8,1023.0,...,NaN,NaN,NaN,NaN,NaN,-4.5,-3.2,-1.3,0.1,0.8


In [31]:
# datetime in '16 data has a double digits for hour
temp_16.head(2)

,datetime,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,dewpoint_temp,spot_pressure,sea_level_pressure,...,snow_3hrs,cloud_amt,mid_cloud_amt,cloud_height,visibility,surface_temp,soil_temp(5cm),soil_temp(10cm),soil_temp(20cm),soil_temp(30cm)
0,2016-01-01 00:00,-6.9,NaN,6.1,290.0,31.0,1.1,-21.2,1011.6,1022.8,...,NaN,0.0,0.0,NaN,2000.0,-3.8,-2.7,-1.1,0.1,0.7
1,2016-01-01 01:00,-7.4,NaN,4.7,290.0,33.0,1.2,-20.9,1011.8,1023.0,...,NaN,NaN,NaN,NaN,NaN,-4.5,-3.2,-1.3,0.1,0.8


In [32]:
for df in [temp_15, temp_17]:
    df['year'] = df['datetime'].apply(lambda x: x[:4]).astype(np.int16)
    df['month'] = df['datetime'].apply(lambda x: x[5:7]).astype(np.int8)
    df['day'] = df['datetime'].apply(lambda x: x[8:10]).astype(np.int8)
    df['hour'] = df['datetime'].apply(lambda x: x[11:13])
    df['hour'].replace(':', '', inplace = True, regex = True)
    df['hour'].astype(np.int8)
    df['date_time'] = df['year'].astype(str) + "-" + df['month'].astype(str) + "-" + df['day'].astype(str) + "-" + df["hour"].astype(str)
    df.drop(['datetime'], axis = 1, inplace = True)

In [33]:
for df in [temp_16]:
    df['year'] = df['datetime'].apply(lambda x: x[:4]).astype(np.int16)
    df['month'] = df['datetime'].apply(lambda x: x[5:7]).astype(np.int8)
    df['day'] = df['datetime'].apply(lambda x: x[8:10]).astype(np.int8)
    df['hour'] = df['datetime'].apply(lambda x: x[11:13]).astype(np.int8)
    df['date_time'] = df['year'].astype(str) + "-" + df['month'].astype(str) + "-" + df['day'].astype(str) + "-" + df["hour"].astype(str)
    df.drop(['datetime'], axis = 1, inplace = True)

### Finally, it's time to fill in NaN values for temp DataFrames 

In [34]:
# Below is the example of columns with NaN Values
temp_15.isnull().any()

temp                  False
rain_amt               True
windspeed             False
wind_direction        False
humidity              False
vapor_pressure        False
dewpoint_temp         False
spot_pressure         False
sea_level_pressure    False
snow_amt               True
snow_3hrs              True
cloud_amt              True
mid_cloud_amt          True
cloud_height           True
visibility             True
surface_temp           True
soil_temp(5cm)         True
soil_temp(10cm)        True
soil_temp(20cm)        True
soil_temp(30cm)        True
year                  False
month                 False
day                   False
hour                  False
date_time             False
dtype: bool

In [35]:
for temp in [temp_15, temp_16, temp_17]:
    # for rain and snow, '0' is a reasonable way to fill in NA
    for col in ['rain_amt', 'snow_amt', 'snow_3hrs']:
        temp[col].fillna(0, inplace = True)
    # for other columns, ffill or bfill sounds more reasonalbe since '0' because sudden drop to 0 doesn't make sense
    for col in ['temp','cloud_amt', 'windspeed', 'wind_direction', 'spot_pressure', 'mid_cloud_amt', 'cloud_height',
                'visibility', 'surface_temp','soil_temp(5cm)','soil_temp(10cm)', 'humidity', 'vapor_pressure',
               'soil_temp(20cm)', 'soil_temp(30cm)', 'dewpoint_temp', 'sea_level_pressure']:
        temp[col].fillna(method = 'ffill', inplace = True)
    # some columns may still have the NaN if prior value is also missing or if the index is 0
    if temp.isnull().values.any():
         for col in ['temp','cloud_amt', 'windspeed', 'wind_direction', 'spot_pressure', 'mid_cloud_amt', 'cloud_height',
                     'visibility', 'surface_temp','soil_temp(5cm)','soil_temp(10cm)', 'humidity', 'vapor_pressure',
               'soil_temp(20cm)', 'soil_temp(30cm)', 'dewpoint_temp', 'sea_level_pressure']:
                temp[col].fillna(method = 'bfill', inplace = True)   
                

In [36]:
print (temp_15.isnull().values.any())
print (temp_16.isnull().values.any())
print (temp_17.isnull().values.any())

False
False
False


#### All Set!

## 3. Prepare the Korean Air Pollution Data

In [37]:
import requests
from bs4 import BeautifulSoup
import json

#### Korean public open data website provides API as below. 

In [38]:
res = requests.get('http://openAPI.seoul.go.kr:8088/554945626b6a696e32364572725249/json/TimeAverageCityAir/1/5/201401020100/도심권/종로구')
data = json.loads(res.content)
data

{'TimeAverageCityAir': {'RESULT': {'CODE': 'INFO-000',
   'MESSAGE': '정상 처리되었습니다'},
  'list_total_count': 1,
  'row': [{'CO': 0.5,
    'MSRADM': '111123',
    'MSRDT': '201401020100',
    'MSRRGN_CD': '100',
    'MSRRGN_NM': '도심권',
    'MSRSTE_NM': '종로구',
    'NO2': 0.027,
    'O3': 0.009,
    'PM10': 93.0,
    'PM24': 97.0,
    'PM25': 22.0,
    'SO2': 0.007}]}}

#### I have downloaded '15, '16, '17 data as below. 
#### Because the process takes a whole amount of time, I saved the Pollution Dataframe as a 'pickle' object for the easy access

In [39]:
# year = ['2015', '2016', '2017']
# month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
# day = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16',
#        '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
# hour = ['00','01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17',
#         '18', '19', '20', '21', '22', '23']


# pollution = pd.DataFrame(columns = ['CO', 'MSRADM', 'MSRDT', 'MSRRGN_CD', 'MSRRGN_NM', 'MSRSTE_NM', 
#                                    'NO2', 'O3', 'PM10', 'PM24', 'PM25', 'SO2'])
# row = 0
# for y in year:
#     print (y)
#     for m in month:
#         for d in day:
#             for h in hour:
#                 row += 1
#                 res = requests.get('http://openAPI.seoul.go.kr:8088/554945626b6a696e32364572725249/json/TimeAverageCityAir/1/5/{}{}{}{}00/도심권/종로구'.format(y, m, d, h))
#                 data = json.loads(res.content)
#                 if 'TimeAverageCityAir' in data:
#                     for key in data['TimeAverageCityAir']['row'][0]:
#                         pollution.loc[row, key] = data['TimeAverageCityAir']['row'][0][key]

In [40]:
# pollution.tail()

In [41]:
import pickle

In [42]:
# pickle.dump(pollution, open('pollution.p', 'wb'))

##### saving to the csv file for backup

In [43]:
# pollution.to_csv('pollution_csv.csv')

In [44]:
pollution = pickle.load(open('pollution.p', 'rb'))

In [45]:
pollution.shape

(22297, 12)

#### Note that the row number of pollution is again, different from the other tables(which must be 22,632 in total; pollution data has only 22,297) 
#### To find out where we are missing the data, let's first break down the table into 3 small tables based on year

In [46]:
# Last row for '16
pollution[pollution['MSRDT'] == '201612312300']

,CO,MSRADM,MSRDT,MSRRGN_CD,MSRRGN_NM,MSRSTE_NM,NO2,O3,PM10,PM24,PM25,SO2
17856,1.3,111123,201612312300,100,도심권,종로구,0.067,0.002,73,63,61,0.005


In [47]:
# Last row for '15
pollution[pollution['MSRDT'] == '201512312300']

,CO,MSRADM,MSRDT,MSRRGN_CD,MSRRGN_NM,MSRSTE_NM,NO2,O3,PM10,PM24,PM25,SO2
8928,0.7,111123,201512312300,100,도심권,종로구,0.052,0.003,89,73,70,0.006


In [48]:
# Dividing pollution DF into three DFs
pollution_15 = pollution.loc[:8928, :]
pollution_16 = pollution.loc[8929:17856, :]
pollution_17 = pollution.loc[17857:, :]

In [49]:
print (pollution_15.shape)
print (pollution_16.shape)
print (pollution_17.shape)

(8753, 12)
(8782, 12)
(4762, 12)


#### Unfortunately, there are missing data for every year
* '15: (-7) data points
* '16: (-2) data points
* '17: (+418) data points

#### '17 data might be a quick fix since the data includes July and August data points that Chinese air and Korean temp data do not have.

In [50]:
pollution_17[pollution_17['MSRDT']=='201706302300']

,CO,MSRADM,MSRDT,MSRRGN_CD,MSRRGN_NM,MSRSTE_NM,NO2,O3,PM10,PM24,PM25,SO2
22296,0.5,111123,201706302300,100,도심권,종로구,0.024,0.038,38,43,31,0.003


In [51]:
pollution_17 = pollution_17.loc[:22296, :]

In [52]:
pollution_17.shape

(4344, 12)

#### The row number matches for '17 now

#### Breaking down the date into year, month, day, hour will help us to align the data with other tables

In [53]:
for df in [pollution_15, pollution_16, pollution_17]:
    df.loc[:,'year'] = df['MSRDT'].apply(lambda x: x[:4]).astype(np.int16)
    df.loc[:,'month'] = df['MSRDT'].apply(lambda x: x[4:6]).astype(np.int8)
    df.loc[:,'day'] = df['MSRDT'].apply(lambda x: x[6:8]).astype(np.int8)
    df.loc[:,'hour'] = df['MSRDT'].apply(lambda x: x[8:10]).astype(np.int8)

/Users/aceit/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/aceit/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [54]:
pollution_15.head()

,CO,MSRADM,MSRDT,MSRRGN_CD,MSRRGN_NM,MSRSTE_NM,NO2,O3,PM10,PM24,PM25,SO2,year,month,day,hour
1,0.2,111123,201501010000,100,도심권,종로구,0.01,0.02,51,42,3,0.005,2015,1,1,0
2,0.2,111123,201501010100,100,도심권,종로구,0.009,0.02,57,45,0,0.004,2015,1,1,1
3,0.2,111123,201501010200,100,도심권,종로구,0.008,0.019,70,49,3,0.005,2015,1,1,2
4,0.2,111123,201501010300,100,도심권,종로구,0.006,0.02,92,57,5,0.005,2015,1,1,3
5,0.2,111123,201501010400,100,도심권,종로구,0.005,0.019,111,65,2,0.004,2015,1,1,4


#### Let's compare pollution_15 and temp_15 data

In [55]:
for i in range(1,13):
    print ("Month: ", i, "Count: ", len(pollution_15[pollution_15['month']==i].index))

Month:  1 Count:  744
Month:  2 Count:  672
Month:  3 Count:  744
Month:  4 Count:  716
Month:  5 Count:  744
Month:  6 Count:  720
Month:  7 Count:  743
Month:  8 Count:  743
Month:  9 Count:  719
Month:  10 Count:  744
Month:  11 Count:  720
Month:  12 Count:  744


In [56]:
for i in range(1,13):
    print ("Month: ", i, "Count: ", len(temp_15[temp_15['month']==i].index))

Month:  1 Count:  744
Month:  2 Count:  672
Month:  3 Count:  744
Month:  4 Count:  720
Month:  5 Count:  744
Month:  6 Count:  720
Month:  7 Count:  744
Month:  8 Count:  744
Month:  9 Count:  720
Month:  10 Count:  744
Month:  11 Count:  720
Month:  12 Count:  744


#### "4" missing data on April,  "1" missing data on July, "1" missing data on August, "1" missing data on September

In [57]:
pollution_15[pollution_15['month'] == 4]['day'].value_counts().sort_index()

1     24
2     24
3     20
4     24
5     24
6     24
7     24
8     24
9     24
10    24
11    24
12    24
13    24
14    24
15    24
16    24
17    24
18    24
19    24
20    24
21    24
22    24
23    24
24    24
25    24
26    24
27    24
28    24
29    24
30    24
Name: day, dtype: int64

#### We can now see that 3rd of April has 4 missing data.  Below is the date and time with missing data, and I have dedcided to interpolate with mean of forward and backward data since the gap is only an hour.
#### '15
    * 4/3: 9, 10, 11, 12
    * 7/24: 15
    * 8/24: 0
    * 9/3: 12
#### '16
    * 9/30: 23
    * 10/1: 0

In [58]:
pollution_15 = pollution_15.reset_index(drop=True)
pollution_16 = pollution_16.reset_index(drop=True)
pollution_17 = pollution_17.reset_index(drop=True)

In [59]:
pollution_15[pollution_15['MSRDT']=='201504030800']

,CO,MSRADM,MSRDT,MSRRGN_CD,MSRRGN_NM,MSRSTE_NM,NO2,O3,PM10,PM24,PM25,SO2,year,month,day,hour
2216,0.6,111123,201504030800,100,도심권,종로구,0.031,0.007,50,23,24,0.006,2015,4,3,8


In [60]:
pollution_15.iloc[2216:2218]

,CO,MSRADM,MSRDT,MSRRGN_CD,MSRRGN_NM,MSRSTE_NM,NO2,O3,PM10,PM24,PM25,SO2,year,month,day,hour
2216,0.6,111123,201504030800,100,도심권,종로구,0.031,0.007,50,23,24,0.006,2015,4,3,8
2217,0,111123,201504031300,100,도심권,종로구,0,0,0,0,0,0,2015,4,3,13


#### From above, we can see that 9, 10, 11, 12 hours are missing on 4/3.
#### I follwed the similar steps to find the index of missing data and inserted the rows as following:

In [61]:
f1 = pd.Series([0.6, 111123, '201504030900', 100, '도심권', '종로구', 0.031, 0.007, 50, 23, 24, 0.006, 2015, 4, 3, 9], 
                index = pollution_15.columns)
f2 = pd.Series([0.6, 111123, '201504031000', 100, '도심권', '종로구', 0.031, 0.007, 50, 23, 24, 0.006, 2015, 4, 3, 10], 
                index = pollution_15.columns)
f3 = pd.Series([0.6, 111123, '201504031100', 100, '도심권', '종로구', 0.031, 0.007, 50, 23, 24, 0.006, 2015, 4, 3, 11], 
                index = pollution_15.columns)
f4 = pd.Series([0.6, 111123, '201504031200', 100, '도심권', '종로구', 0.031, 0.007, 50, 23, 24, 0.006, 2015, 4, 3, 12], 
                index = pollution_15.columns)
f5 = pd.Series([0.5, 111123, '201507241500', 100, '도심권', '종로구', 0.036, 0.012, 18, 15, 13, 0.007, 2015, 7, 24, 15], 
                index = pollution_15.columns)
f6 = pd.Series([0.4, 111123, '201508240000', 100, '도심권', '종로구', 0.021, 0.029, 37, 35, 24, 0.005, 2015, 8, 24, 0], 
                index = pollution_15.columns)
f7 = pd.Series([0.4, 111123, '201509031200', 100, '도심권', '종로구', 0.016, 0.053, 17, 21, 0, 0.007, 2015, 9, 3, 12], 
                index = pollution_15.columns)
df = pd.DataFrame([f1, f2, f3, f4, f5, f6, f7], columns = pollution_15.columns)
pollution_15 = pd.concat([pollution_15, df], axis = 0)
del ([f1, f2, f3, f4, f5, f6, f7, df])

In [62]:
f1 = pd.Series([0, 111123, '201609302300', 100, '도심권', '종로구', 0, 0, 0, 0, 0, 0, 2016, 9, 30, 23], 
                index = pollution_16.columns)
f2 = pd.Series([0, 111123, '201610010000', 100, '도심권', '종로구', 0, 0, 0, 0, 0, 0, 2016, 10, 1, 0], 
                index = pollution_16.columns)
df = pd.DataFrame([f1, f2], columns = pollution_16.columns)
pollution_16 = pd.concat([pollution_16, df], axis = 0)
del ([f1, f2, df])

In [63]:
pollution_15.sort_values('MSRDT', inplace = True)
pollution_15 = pollution_15.reset_index(drop=True)
pollution_16.sort_values('MSRDT', inplace = True)
pollution_16 = pollution_16.reset_index(drop=True)

In [64]:
pollution_15.tail()

,CO,MSRADM,MSRDT,MSRRGN_CD,MSRRGN_NM,MSRSTE_NM,NO2,O3,PM10,PM24,PM25,SO2,year,month,day,hour
8755,0.7,111123,201512311900,100,도심권,종로구,0.048,0.005,85,64,67,0.006,2015,12,31,19
8756,0.8,111123,201512312000,100,도심권,종로구,0.051,0.004,96,66,75,0.006,2015,12,31,20
8757,0.8,111123,201512312100,100,도심권,종로구,0.051,0.003,95,69,75,0.006,2015,12,31,21
8758,0.7,111123,201512312200,100,도심권,종로구,0.05,0.003,92,71,73,0.006,2015,12,31,22
8759,0.7,111123,201512312300,100,도심권,종로구,0.052,0.003,89,73,70,0.006,2015,12,31,23


In [67]:
print (pollution_15.shape[0], pollution_16.shape[0], pollution_17.shape[0])

8760 8784 4344


### The row size has been corrected :)
<br>
<br>
### Now, let's clean up the table to finalize

In [68]:
for df in [pollution_15, pollution_16, pollution_17]:
    df['date_time'] = df['date_time'] = df['year'].astype(str) + "-" + df['month'].astype(str) + "-" + df['day'].astype(str) + "-" + df["hour"].astype(str)
    df.drop(['MSRADM', 'MSRDT', 'MSRRGN_CD', 'MSRRGN_NM', 'MSRSTE_NM', 'year', 'month', 'day', 'hour'], 
            axis = 1, inplace = True)

In [69]:
pollution_15.head(2)

,CO,NO2,O3,PM10,PM24,PM25,SO2,date_time
0,0.2,0.01,0.02,51,42,3,0.005,2015-1-1-0
1,0.2,0.009,0.02,57,45,0,0.004,2015-1-1-1


#### Making sure the values are in numeric format for numpy calculation

In [70]:
for col in  pollution_15.columns[:-1]:
    pollution_15[col] = pd.to_numeric(pollution_15[col], errors='coerce')

In [71]:
for col in  pollution_16.columns[:-1]:
    pollution_16[col] = pd.to_numeric(pollution_16[col], errors='coerce')

In [72]:
for col in  pollution_17.columns[:-1]:
    pollution_17[col] = pd.to_numeric(pollution_17[col], errors='coerce')

## 4. Creating the comprehensive DataFrame

#### Now I have multiple DataFrames with the same number of rows, and it's time to create two comprehensive DataFrames: Train DF and Test DF

#### I am going to use '15 and '16 data as training sets and use '17 data as a test set

In [73]:
for df in [beijing_15, beijing_16, beijing_17, shanghai_15, 
           shanghai_16, shanghai_17, shenyang_15, shenyang_16, shenyang_17]:
    df.reset_index(drop=True, inplace =True)

#### While I was trying to merge all the dataframes, I have found the miss typos such as duplicate dates or unrealistic dates.  Let's clean them up first before merging the dataframes

In [74]:
beijing_15.loc[1586, 'date_time'] = '2015-3-8-2'
shanghai_15.loc[1586, 'date_time'] = '2015-3-8-2'
shenyang_15.loc[1586, 'date_time'] = '2015-3-8-2'

beijing_16.loc[1727, 'date_time'] = '2016-3-12-23'
shanghai_16.loc[1727, 'date_time'] = '2016-3-12-23'
shenyang_16.loc[1727, 'date_time'] = '2016-3-12-23'
beijing_16.loc[1730, 'date_time'] = '2016-3-13-2'
shanghai_16.loc[1730, 'date_time'] = '2016-3-13-2'
shenyang_16.loc[1730, 'date_time'] = '2016-3-13-2'

beijing_17.loc[1682, 'date_time'] = '2017-3-12-2'
shanghai_17.loc[1682, 'date_time'] = '2017-3-12-2'
shenyang_17.loc[1682, 'date_time'] = '2017-3-12-2'


In [75]:
df_15 = beijing_15.merge(shanghai_15, on='date_time').merge(shenyang_15, 
                        on='date_time').merge(temp_15, 
                                        on = 'date_time').merge(pollution_15, on='date_time')

In [76]:
print (df_15.shape)
df_15.head(2)

(8760, 35)


,Value_Beijing,date_time,Value_Shanghai,Value_Shenyang,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,...,month,day,hour,CO,NO2,O3,PM10,PM24,PM25,SO2
0,22.0,2015-1-1-0,32.0,37.0,-6.9,0.0,6.1,290,31.0,1.1,...,1,1,0,0.2,0.010,0.02,51.0,42.0,3.0,0.005
1,9.0,2015-1-1-1,40.0,26.0,-7.4,0.0,4.7,290,33.0,1.2,...,1,1,1,0.2,0.009,0.02,57.0,45.0,0.0,0.004


In [77]:
df_16 = beijing_16.merge(shanghai_16, on='date_time').merge(shenyang_16, 
                        on='date_time').merge(temp_16, 
                                        on = 'date_time').merge(pollution_16, on='date_time')

In [78]:
print (df_16.shape)
df_16.head(2)

(8784, 35)


,Value_Beijing,date_time,Value_Shanghai,Value_Shenyang,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,...,month,day,hour,CO,NO2,O3,PM10,PM24,PM25,SO2
0,231.0,2016-1-1-0,60.0,218.0,-6.9,0.0,6.1,290.0,31.0,1.1,...,1,1,0,0.8,0.057,0.002,87.0,74.0,68.0,0.006
1,239.0,2016-1-1-1,65.0,170.0,-7.4,0.0,4.7,290.0,33.0,1.2,...,1,1,1,1.1,0.066,0.002,84.0,74.0,63.0,0.006


In [79]:
df_17 = beijing_17.merge(shanghai_17, on='date_time').merge(shenyang_17, 
                        on='date_time').merge(temp_17, 
                                        on = 'date_time').merge(pollution_17, on='date_time')

In [80]:
print (df_17.shape)
df_17.head(2)

(4344, 35)


,Value_Beijing,date_time,Value_Shanghai,Value_Shenyang,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,...,month,day,hour,CO,NO2,O3,PM10,PM24,PM25,SO2
0,505.0,2017-1-1-0,42.0,120.0,-6.9,0.0,6.1,290,31.0,1.1,...,1,1,0,1.3,0.065,0.002,72.0,64.0,60.0,0.004
1,485.0,2017-1-1-1,46.0,121.0,-7.4,0.0,4.7,290,33.0,1.2,...,1,1,1,1.2,0.059,0.002,73.0,63.0,57.0,0.004


In [81]:
df_17[df_17['date_time'] == '2017-3-12-3']

,Value_Beijing,date_time,Value_Shanghai,Value_Shenyang,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,...,month,day,hour,CO,NO2,O3,PM10,PM24,PM25,SO2
1683,11.0,2017-3-12-3,53.0,23.0,-2.1,0.0,4.0,250,47.0,2.5,...,3,12,3,1.1,0.086,0.002,84.0,71.0,68.0,0.006


In [84]:
train_set = pd.concat([df_15, df_16], axis = 0)

In [85]:
train_set.tail()

,Value_Beijing,date_time,Value_Shanghai,Value_Shenyang,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,...,month,day,hour,CO,NO2,O3,PM10,PM24,PM25,SO2
8779,409.0,2016-12-31-19,43.0,82.0,-1.6,0.0,0.8,50.0,85.0,4.6,...,12,31,19,1.0,0.057,0.002,66.0,64.0,53.0,0.005
8780,432.0,2016-12-31-20,40.0,82.0,-1.6,0.0,0.8,50.0,85.0,4.6,...,12,31,20,1.4,0.069,0.002,74.0,64.0,58.0,0.005
8781,482.0,2016-12-31-21,35.0,82.0,-1.6,0.0,0.8,50.0,85.0,4.6,...,12,31,21,1.5,0.073,0.002,79.0,64.0,63.0,0.005
8782,488.0,2016-12-31-22,39.0,82.0,-1.6,0.0,0.8,50.0,85.0,4.6,...,12,31,22,1.5,0.072,0.002,76.0,64.0,61.0,0.005
8783,507.0,2016-12-31-23,44.0,82.0,-1.6,0.0,0.8,50.0,85.0,4.6,...,12,31,23,1.3,0.067,0.002,73.0,63.0,61.0,0.005


#### Finally, I will change the date_time format to the Timestamp and the template is all done
#### This template will be used for the correlation analysis 

In [86]:
train_set_template = (train_set
                 .assign(date_time = lambda x: x.date_time.map(lambda x: datetime.strptime(x, '%Y-%m-%d-%H')))
                )

In [87]:
test_set_template = (df_17.assign(date_time = lambda x: x.date_time.map(lambda x: datetime.strptime(x, '%Y-%m-%d-%H'))))

In [88]:
print (train_set_template.shape)
train_set_template.head()

(17544, 35)


,Value_Beijing,date_time,Value_Shanghai,Value_Shenyang,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,...,month,day,hour,CO,NO2,O3,PM10,PM24,PM25,SO2
0,22.0,2015-01-01 00:00:00,32.0,37.0,-6.9,0.0,6.1,290.0,31.0,1.1,...,1,1,0,0.2,0.010,0.020,51.0,42.0,3.0,0.005
1,9.0,2015-01-01 01:00:00,40.0,26.0,-7.4,0.0,4.7,290.0,33.0,1.2,...,1,1,1,0.2,0.009,0.020,57.0,45.0,0.0,0.004
2,9.0,2015-01-01 02:00:00,37.0,30.0,-8.0,0.0,4.5,290.0,35.0,1.2,...,1,1,2,0.2,0.008,0.019,70.0,49.0,3.0,0.005
3,13.0,2015-01-01 03:00:00,44.0,26.0,-8.4,0.0,3.8,290.0,37.0,1.2,...,1,1,3,0.2,0.006,0.020,92.0,57.0,5.0,0.005
4,10.0,2015-01-01 04:00:00,38.0,33.0,-8.8,0.0,4.9,290.0,35.0,1.1,...,1,1,4,0.2,0.005,0.019,111.0,65.0,2.0,0.004


In [89]:
print (test_set_template.shape)
test_set_template.head()

(4344, 35)


,Value_Beijing,date_time,Value_Shanghai,Value_Shenyang,temp,rain_amt,windspeed,wind_direction,humidity,vapor_pressure,...,month,day,hour,CO,NO2,O3,PM10,PM24,PM25,SO2
0,505.0,2017-01-01 00:00:00,42.0,120.0,-6.9,0.0,6.1,290,31.0,1.1,...,1,1,0,1.3,0.065,0.002,72.0,64.0,60.0,0.004
1,485.0,2017-01-01 01:00:00,46.0,121.0,-7.4,0.0,4.7,290,33.0,1.2,...,1,1,1,1.2,0.059,0.002,73.0,63.0,57.0,0.004
2,466.0,2017-01-01 02:00:00,56.0,127.0,-8.0,0.0,4.5,290,35.0,1.2,...,1,1,2,1.2,0.058,0.002,71.0,63.0,59.0,0.004
3,435.0,2017-01-01 03:00:00,49.0,128.0,-8.4,0.0,3.8,290,37.0,1.2,...,1,1,3,1.2,0.056,0.002,70.0,63.0,59.0,0.004
4,405.0,2017-01-01 04:00:00,51.0,135.0,-8.8,0.0,4.9,290,35.0,1.1,...,1,1,4,1.2,0.056,0.002,70.0,62.0,58.0,0.004


In [90]:
train_set_template.to_csv('train_set_template.csv')
test_set_template.to_csv('test_set_template.csv')

In [91]:
pickle.dump(train_set_template, open('train_set_template.p', 'wb'))
pickle.dump(test_set_template, open('test_set_template.p', 'wb'))

# B: Data preparation for Prediction

#### The template for the correlation analysis is composed of the same date_time data
#### To make a prediction of microdust level, however, the same date data is actaully useless.
#### With the given information of today, I have to make a prediction for future microdust level. And therefore, the template for prediction should be composed of 'weather forecast' and 'today's dust levels' as independent variables, and 'dust level after n-days' as a dependent variable.

In [92]:
# Below csv files are the weather forecast data from National Weather Service
fc_temp_15 = pd.read_csv('forecast/2015_3hr_temp.csv')
fc_temp_15.name = 'fc_temp_15'
fc_humid_15 = pd.read_csv('forecast/2015_humidity.csv')
fc_humid_15.name = 'fc_humid_15'
fc_rain_15 = pd.read_csv('forecast/2015_rain_probability.csv')
fc_rain_15.name = 'fc_rain_15'
fc_snow_15 = pd.read_csv('forecast/2015_snow_6hrs.csv')
fc_snow_15.name = 'fc_snow_15'
fc_wind_d_15 = pd.read_csv('forecast/2015_wind_direction.csv')
fc_wind_d_15.name = 'fc_wind_d_15'
fc_wind_s_15 = pd.read_csv('forecast/2015_wind_speed.csv')
fc_wind_s_15.name = 'fc_wind_s_15'

In [93]:
fc_temp_15.head()

,format: day,hour,forecast,value location:60_127 Start : 20150101
0,1,200.0,4.0,-5.0
1,1,200.0,7.0,-6.0
2,1,200.0,10.0,-7.0
3,1,200.0,13.0,-8.0
4,1,200.0,16.0,-8.0


In [94]:
for i in [fc_temp_15, fc_humid_15, fc_rain_15, fc_wind_d_15, fc_wind_s_15]:
    print (i.shape)

(52530, 4)
(52530, 4)
(52530, 4)
(52530, 4)
(52530, 4)


#### The forecast data has prediction values up to 67 hours ahead in 3 hours frequencies.
#### I would like to predict the dust level for post 2 days, so I will get rid of other rows except '46', which is the cloesest to '48hours'(2days)

In [95]:
for i in [fc_temp_15, fc_humid_15, fc_rain_15, fc_wind_d_15, fc_wind_s_15]:
    i.insert(4, i.name, i.iloc[:,3])
    i.drop(i[i['forecast'] != 46.0].index, inplace = True)
    i.reset_index(drop = True, inplace = True)

#### Concat the values from multiple weather DataFrames

In [96]:
fc_15 = pd.concat([fc_temp_15[[' format: day', 'hour', 'forecast', 'fc_temp_15']], fc_humid_15['fc_humid_15'],
          fc_rain_15['fc_rain_15'], fc_wind_d_15['fc_wind_d_15'], fc_wind_s_15['fc_wind_s_15']], axis = 1)

In [97]:
fc_15.head()

,format: day,hour,forecast,fc_temp_15,fc_humid_15,fc_rain_15,fc_wind_d_15,fc_wind_s_15
0,1,200.0,46.0,-6.0,50.0,0.0,102.0,1.0
1,1,500.0,46.0,-1.0,45.0,10.0,129.0,2.0
2,1,800.0,46.0,1.0,50.0,20.0,159.0,2.0
3,1,1100.0,46.0,0.0,55.0,20.0,150.0,2.0
4,1,1400.0,46.0,0.0,65.0,20.0,146.0,2.0


#### I also need to create the date_time column, which I can use as an index when I merge DataFrames on later step. 
#### Creating the 'month' information was the most tricky part

In [98]:
fc_15.rename(columns = {' format: day' : 'day'}, inplace = True)
m = 1
fc_15['month'] = 1
for i in range(fc_15.shape[0]-1):
    if int(fc_15.loc[i+1, 'day']) - int(fc_15.loc[i, 'day']) < 0:
        m += 1
        fc_15.loc[i+1, 'month'] = m
    else:
        fc_15.loc[i+1, 'month'] = m
        
fc_15['hour'] = fc_15['hour'].astype(int) // 100
fc_15.head()

,day,hour,forecast,fc_temp_15,fc_humid_15,fc_rain_15,fc_wind_d_15,fc_wind_s_15,month
0,1,2,46.0,-6.0,50.0,0.0,102.0,1.0,1
1,1,5,46.0,-1.0,45.0,10.0,129.0,2.0,1
2,1,8,46.0,1.0,50.0,20.0,159.0,2.0,1
3,1,11,46.0,0.0,55.0,20.0,150.0,2.0,1
4,1,14,46.0,0.0,65.0,20.0,146.0,2.0,1


In [99]:
fc_15['date_time'] = '2015' + "-" + fc_15['month'].astype(str) + "-" + fc_15['day'].astype(str) + "-" + fc_15["hour"].astype(str)
fc_15.date_time.replace({' ': ''}, regex=True, inplace = True)
fc_15 = (fc_15.assign(date_time = lambda x: x.date_time.map(lambda x: datetime.strptime(x, '%Y-%m-%d-%H'))))

In [100]:
fc_15.iloc[245:251]

,day,hour,forecast,fc_temp_15,fc_humid_15,fc_rain_15,fc_wind_d_15,fc_wind_s_15,month,date_time
245,31,17,46.0,0.0,65.0,0.0,225.0,2.0,1,2015-01-31 17:00:00
246,31,20,46.0,-1.0,65.0,0.0,130.0,1.0,1,2015-01-31 20:00:00
247,31,23,46.0,-2.0,65.0,10.0,204.0,1.0,1,2015-01-31 23:00:00
248,1,2,46.0,0.0,55.0,20.0,72.0,1.0,2,2015-02-01 02:00:00
249,1,5,46.0,3.0,40.0,20.0,144.0,2.0,2,2015-02-01 05:00:00
250,1,8,46.0,4.0,35.0,20.0,249.0,2.0,2,2015-02-01 08:00:00


In [101]:
fc_15.shape

(2839, 10)

#### And the rest is just repeating the same steps for '16 and '17.  I could have defied a function and have used it, but to make sure I have the right shape and structure for each DataFrame, a seperate operation was chosen.

In [102]:
fc_temp_16 = pd.read_csv('forecast/2016_3hr_temp(4-12).csv')
fc_temp_16.name = 'fc_temp_16'
fc_humid_16 = pd.read_csv('forecast/2016_humidity(4-12).csv')
fc_humid_16.name = 'fc_humid_16'
fc_rain_16 = pd.read_csv('forecast/2016_rain_probability(4-12).csv')
fc_rain_16.name = 'fc_rain_16'
fc_snow_16 = pd.read_csv('forecast/2016_snow_6hrs(4-12).csv')
fc_snow_16.name = 'fc_snow_16'
fc_wind_d_16 = pd.read_csv('forecast/2016_wind_direction(4-12).csv')
fc_wind_d_16.name = 'fc_wind_d_16'
fc_wind_s_16 = pd.read_csv('forecast/2016_wind_speed(4-12).csv')
fc_wind_s_16.name = 'fc_wind_s_16'

for i in [fc_temp_16, fc_humid_16, fc_rain_16, fc_wind_d_16, fc_wind_s_16]:
    print (i.shape)

(54179, 4)
(54179, 4)
(54179, 4)
(54179, 4)
(54179, 4)


In [103]:
for i in [fc_temp_16, fc_humid_16, fc_rain_16, fc_wind_d_16, fc_wind_s_16]:
    i.insert(4, i.name, i.iloc[:,3])
    i.drop(i[i['forecast'] != 46.0].index, inplace = True)
    i.reset_index(drop = True, inplace = True)
    
fc_16 = pd.concat([fc_temp_16[[' format: day', 'hour', 'forecast', 'fc_temp_16']], fc_humid_16['fc_humid_16'],
          fc_rain_16['fc_rain_16'], fc_wind_d_16['fc_wind_d_16'], fc_wind_s_16['fc_wind_s_16']], axis = 1)

fc_16.head()

,format: day,hour,forecast,fc_temp_16,fc_humid_16,fc_rain_16,fc_wind_d_16,fc_wind_s_16
0,1,200.0,46.0,4.0,70.0,20.0,300.0,1.0
1,1,500.0,46.0,6.0,55.0,20.0,292.0,2.0
2,1,800.0,46.0,7.0,40.0,20.0,287.0,3.0
3,1,1100.0,46.0,5.0,50.0,20.0,298.0,2.0
4,1,1400.0,46.0,3.0,65.0,20.0,321.0,3.0


In [104]:
fc_16.rename(columns = {' format: day' : 'day'}, inplace=True)
m = 1
fc_16['month'] = 1
for i in range(fc_16.shape[0]-1):
    if int(fc_16.loc[i+1, 'day']) - int(fc_16.loc[i, 'day']) < 0:
        m += 1
        fc_16.loc[i+1, 'month'] = m
    else:
        fc_16.loc[i+1, 'month'] = m
        
fc_16['hour'] = fc_16['hour'].astype(int) // 100
fc_16.head()

,day,hour,forecast,fc_temp_16,fc_humid_16,fc_rain_16,fc_wind_d_16,fc_wind_s_16,month
0,1,2,46.0,4.0,70.0,20.0,300.0,1.0,1
1,1,5,46.0,6.0,55.0,20.0,292.0,2.0,1
2,1,8,46.0,7.0,40.0,20.0,287.0,3.0,1
3,1,11,46.0,5.0,50.0,20.0,298.0,2.0,1
4,1,14,46.0,3.0,65.0,20.0,321.0,3.0,1


In [105]:
fc_16['date_time'] = '2016' + "-" + fc_16['month'].astype(str) + "-" + fc_16['day'].astype(str) + "-" + fc_16["hour"].astype(str)
fc_16.date_time.replace({' ': ''}, regex=True, inplace = True)
fc_16 = (fc_16.assign(date_time = lambda x: x.date_time.map(lambda x: datetime.strptime(x, '%Y-%m-%d-%H'))))
fc_16.head()

,day,hour,forecast,fc_temp_16,fc_humid_16,fc_rain_16,fc_wind_d_16,fc_wind_s_16,month,date_time
0,1,2,46.0,4.0,70.0,20.0,300.0,1.0,1,2016-01-01 02:00:00
1,1,5,46.0,6.0,55.0,20.0,292.0,2.0,1,2016-01-01 05:00:00
2,1,8,46.0,7.0,40.0,20.0,287.0,3.0,1,2016-01-01 08:00:00
3,1,11,46.0,5.0,50.0,20.0,298.0,2.0,1,2016-01-01 11:00:00
4,1,14,46.0,3.0,65.0,20.0,321.0,3.0,1,2016-01-01 14:00:00


In [106]:
fc_15.columns

Index(['day', 'hour', 'forecast', 'fc_temp_15', 'fc_humid_15', 'fc_rain_15',
       'fc_wind_d_15', 'fc_wind_s_15', 'month', 'date_time'],
      dtype='object')

In [107]:
fc_15.columns = [['day', 'hour', 'forecast', 'fc_temp', 'fc_humid', 'fc_rain',
       'fc_wind_d', 'fc_wind_s', 'month', 'date_time']]
fc_16.columns = [['day', 'hour', 'forecast', 'fc_temp', 'fc_humid', 'fc_rain',
       'fc_wind_d', 'fc_wind_s', 'month', 'date_time']]

#### After concatenating '15 and '16 forecasts, I will name it 'fc_train'

In [108]:
fc_train = pd.concat([fc_15, fc_16], axis = 0)
fc_train.shape

(5767, 10)

In [109]:
fc_train.head()

,day,hour,forecast,fc_temp,fc_humid,fc_rain,fc_wind_d,fc_wind_s,month,date_time
0,1,2,46.0,-6.0,50.0,0.0,102.0,1.0,1,2015-01-01 02:00:00
1,1,5,46.0,-1.0,45.0,10.0,129.0,2.0,1,2015-01-01 05:00:00
2,1,8,46.0,1.0,50.0,20.0,159.0,2.0,1,2015-01-01 08:00:00
3,1,11,46.0,0.0,55.0,20.0,150.0,2.0,1,2015-01-01 11:00:00
4,1,14,46.0,0.0,65.0,20.0,146.0,2.0,1,2015-01-01 14:00:00


In [110]:
fc_train.reset_index(drop = True, inplace = True)

In [111]:
fc_temp_17 = pd.read_csv('forecast/2017_3hr_temp.csv')
fc_temp_17.name = 'fc_temp_17'
fc_humid_17 = pd.read_csv('forecast/2017_humidity.csv')
fc_humid_17.name = 'fc_humid_17'
fc_rain_17 = pd.read_csv('forecast/2017_rain_probability.csv')
fc_rain_17.name = 'fc_rain_17'
fc_snow_17 = pd.read_csv('forecast/2017_snow_6hrs.csv')
fc_snow_17.name = 'fc_snow_17'
fc_wind_d_17 = pd.read_csv('forecast/2017_wind_direction.csv')
fc_wind_d_17.name = 'fc_wind_d_17'
fc_wind_s_17 = pd.read_csv('forecast/2017_wind_speed.csv')
fc_wind_s_17.name = 'fc_wind_s_17'

for i in [fc_temp_17, fc_humid_17, fc_rain_17, fc_wind_d_17, fc_wind_s_17]:
    print (i.shape)

(26793, 4)
(26793, 4)
(26793, 4)
(26793, 4)
(26793, 4)


In [112]:
for i in [fc_temp_17, fc_humid_17, fc_rain_17, fc_wind_d_17, fc_wind_s_17]:
    i.insert(4, i.name, i.iloc[:,3])
    i.drop(i[i['forecast'] != 46.0].index, inplace = True)
    i.reset_index(drop = True, inplace = True)

fc_17 = pd.concat([fc_temp_17[[' format: day', 'hour', 'forecast', 'fc_temp_17']], fc_humid_17['fc_humid_17'],
          fc_rain_17['fc_rain_17'], fc_wind_d_17['fc_wind_d_17'], fc_wind_s_17['fc_wind_s_17']], axis = 1)

fc_17.head()

,format: day,hour,forecast,fc_temp_17,fc_humid_17,fc_rain_17,fc_wind_d_17,fc_wind_s_17
0,1,200.0,46.0,-1.0,70.0,0.0,198.0,1.0
1,1,500.0,46.0,3.0,50.0,0.0,277.0,3.0
2,1,800.0,46.0,5.0,35.0,0.0,256.0,3.0
3,1,1100.0,46.0,3.0,45.0,0.0,225.0,3.0
4,1,1400.0,46.0,1.0,60.0,0.0,178.0,2.0


In [113]:
fc_17.rename(columns = {' format: day' : 'day'}, inplace=True)
m = 1
fc_17['month'] = 1
for i in range(fc_17.shape[0]-1):
    if int(fc_17.loc[i+1, 'day']) - int(fc_17.loc[i, 'day']) < 0:
        m += 1
        fc_17.loc[i+1, 'month'] = m
    else:
        fc_17.loc[i+1, 'month'] = m
        
fc_17['hour'] = fc_17['hour'].astype(int) // 100
fc_17.head()

,day,hour,forecast,fc_temp_17,fc_humid_17,fc_rain_17,fc_wind_d_17,fc_wind_s_17,month
0,1,2,46.0,-1.0,70.0,0.0,198.0,1.0,1
1,1,5,46.0,3.0,50.0,0.0,277.0,3.0,1
2,1,8,46.0,5.0,35.0,0.0,256.0,3.0,1
3,1,11,46.0,3.0,45.0,0.0,225.0,3.0,1
4,1,14,46.0,1.0,60.0,0.0,178.0,2.0,1


In [114]:
fc_17['date_time'] = '2017' + "-" + fc_17['month'].astype(str) + "-" + fc_17['day'].astype(str) + "-" + fc_17["hour"].astype(str)
fc_17.date_time.replace({' ': ''}, regex=True, inplace = True)
fc_17 = (fc_17.assign(date_time = lambda x: x.date_time.map(lambda x: datetime.strptime(x, '%Y-%m-%d-%H'))))

In [115]:
fc_17.columns = [['day', 'hour', 'forecast', 'fc_temp', 'fc_humid', 'fc_rain',
       'fc_wind_d', 'fc_wind_s', 'month', 'date_time']]
fc_test = fc_17

In [116]:
fc_test.head()

,day,hour,forecast,fc_temp,fc_humid,fc_rain,fc_wind_d,fc_wind_s,month,date_time
0,1,2,46.0,-1.0,70.0,0.0,198.0,1.0,1,2017-01-01 02:00:00
1,1,5,46.0,3.0,50.0,0.0,277.0,3.0,1,2017-01-01 05:00:00
2,1,8,46.0,5.0,35.0,0.0,256.0,3.0,1,2017-01-01 08:00:00
3,1,11,46.0,3.0,45.0,0.0,225.0,3.0,1,2017-01-01 11:00:00
4,1,14,46.0,1.0,60.0,0.0,178.0,2.0,1,2017-01-01 14:00:00


In [117]:
pickle.dump(fc_train, open('fc_train.p', 'wb'))
pickle.dump(fc_test, open('fc_test.p', 'wb'))

## Now we have two complete DataFrames that we can use for analysis and prediction.  
### - To be contiued on the other notebook -